# IBM Model 2, Expectation Maximisation
#### Authors: Adriaan de Vries, Féliciën Veldema, Verna Dankers

This notebook implements the expectation maximisation training algorithm for IBM Model 2. Run the cells in order to run the algorithm.

### 1. Requirements

In [1]:
# Python libraries to install
from __future__ import print_function, division
from collections import defaultdict, Counter
from tqdm import tqdm
from random import random
from scipy import special

import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import os

# Custom imports
from aer import read_naacl_alignments, AERSufficientStatistics
from data import read_data

### 2. Read in the data

Please set the paths to the data and run the code below. Functions for reading in the data have been placed outside of the notebook, as they are re-used by other notebooks.

In [2]:
english_train = 'training/hansards.36.2.e'
french_train = 'training/hansards.36.2.f'
english_val = 'validation/dev.e'
french_val = 'validation/dev.f'
fname = 'naacltest.txt'

# Read training data and replace words occurring once by -UNK-
training_data = read_data(english_train, french_train, True)
ext_data = list(zip(*training_data))

# Replace words in validation data that do not appear in the training data
validation_data = read_data(english_val, french_val, True, ttype='validation', eng_data=ext_data[0], fre_data=ext_data[1])

Adding the -UNK- token to the data.
English data complete.
French data complete
Adding the -UNK- token to the data.
English data complete.
French data complete


### 3. Implementation of IBM2 EM

First, we implement the training algorithm, and the functions to calculate alignments and log likelihood.

In [ ]:
def align_all(data, translate_dict, jump_dict, fname=None):
    """Create alignments for pairs of English and French sentences.
    Both save them as sets per sentence and pair and save to file.
    
    Args:
        validation: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f
        fname: filename to save alignments in, in NAACL format

    Returns:
        list of sets
    """
    file = open(fname, 'w')
    alignments = []
    for k, (english_words, french_words) in enumerate(data):
        alignment = align(english_words, french_words, translate_dict, jump_dict, False)
        for pos1, pos2 in alignment:
            file.write("{} {} {}\n".format(str(k+1), str(pos1), str(pos2)))
        alignments.append(set(alignment))
    return alignments
    
def align(english_words, french_words, translate_dict, jump_dict, add_null=True):
    """Align one sentence pair, either with or without the NULL alignments.
    
    Args:
        english_words: list of english words
        french_words: list of french words
        translate_dict: dictionary with translation probabilities e to f
        add_null: boolean to indicate whether NULL alignments should be included

    Return:
        list of tuples
    """
    alignment = []
    for j, fword in enumerate(french_words):
        prior = 0.0
        alignment_j = 0
        for i, eword in enumerate(english_words):
            # Only include terms that are in the dictionary
            if eword in translate_dict and fword in translate_dict[eword]:
                prob = translate_dict[eword][fword] * jump_dict[get_jump(j, i, len(english_words), len(french_words))]
                if prob > prior:
                    prior = prob
                    alignment_j = i
        # Add dependent on whether it's a NULL alignments
        if alignment_j != 0 or add_null:
            alignment.append((alignment_j, j + 1))
    return alignment

def initialize_t(data, uniform=True):
    """Initialise the translation probabilities.
    
    Args:
        data: list of tuples, english and french sentences
        uniform: boolean indicating initialisation type

    Returns:
        defaultdict(Counter)
    """
    co_counts = defaultdict(Counter)
    for e, f in data:
        for e_word in e:
            for f_word in f:
                if uniform:
                    co_counts[e_word][f_word] = 1
                else:
                    co_counts[e_word][f_word] = random()
    for e_word in co_counts:
        normalization_factor = sum(list(co_counts[e_word].values()))
        for f_word in co_counts[e_word]:
            co_counts[e_word][f_word] = co_counts[e_word][f_word] / normalization_factor
    return co_counts

def log_likelihood(data, translate_dict, jump_dict):
    """
    Args:
        data: zipped object with pairs of e and f sentences
        translate_dict: dictionary with translation probabilities e to f

    Returns:
        float: log likelihood
    """
    log_likelihood = 0
    for e, f in data:
        alignment = align(e, f, translate_dict, jump_dict, True)
        logprob = 0
        for i, j in alignment:
            logprob += math.log(translate_dict[e[i]][f[j-1]] * jump_dict[get_jump(j-1, i, len(e), len(f))])
        log_likelihood += logprob
    return log_likelihood

def test(path, personal_sets=None):
    """
    Args:
        path (str): path with naacl alignments
        personal_sets: alignments in set repersesntation
    
    Returns:
        int: AER
    """
    from random import random
    # 1. Read in gold alignments
    gold_sets = read_naacl_alignments('validation/dev.wa.nonullalign')

    # 2. Here you would have the predictions of your own algorithm
    if personal_sets is None:
        personal_sets = read_naacl_alignments(path)
        predictions = []
        for s, p in personal_sets:
            links = set()
            for link in s:
                links.add(link)
            predictions.append(links)
    else:
        predictions=personal_sets

    # 3. Compute AER
    # first we get an object that manages sufficient statistics 
    metric = AERSufficientStatistics()
    # then we iterate over the corpus 
    for gold, pred in zip(gold_sets, predictions):
        metric.update(sure=gold[0], probable=gold[1], predicted=pred)
    # AER
    return metric.aer()

def EM_IBM2(data, validation, initial_translation_estimate=None, initialise_method = 'IBM1', max_steps=3):
    """Train IBM 2 using expectation maximisation.
    
    Args:
        training_data: list of tuples with parallel sentences
        validation: list of tuples with parallel sentences
        alpha: dirichlet prior parametrised
        initial_translation_estimate: translation_dict initialiation
        initialise_method: how to initialise translation probabilities
        max_steps: number of learning iterations

    Returns:
        dict: translation probabilities
        dict: jump probabilities
    """
    initialise_method = initialise_method.lower()
    assert initialise_method in ['ibm1', 'uniform', 'random'], "initialise method has to be in [ibm1, uniform, random]"
    if initialise_method == 'ibm1':
        assert initial_translation_estimate, "initial_translation_method has to be given, if ibm1"
        translate_dict = initial_translation_estimate
    elif initialise_method == 'uniform':
        translate_dict = initialize_t(data, True)
    else:
        translate_dict = initialize_t(data, False)
    
    jump_dict = {}
    for i in range(-100, 101):
        jump_dict[i] = 1/201
    aers = []
    lls = []

    for iteration in range(max_steps):
        fname = 'IBM2_iteration' + str(iteration) + '.txt'
        counts = defaultdict(int)
        co_counts = defaultdict(int)
        jump_counts = defaultdict(int)
        pos_counts = 0
        
        print("Expectation step {}".format(iteration))
        for e_s,f_s in tqdm(data):
            m = len(f_s)
            l = len(e_s)
            for j, f in enumerate(f_s):
                sum_of_probs = 0
                for i, e in enumerate(e_s):
                    jump_prob = jump_dict.get(get_jump(j,i,l,m), 1/l)
                    translate_prob = translate_dict[e][f]
                    sum_of_probs += jump_prob * translate_prob
                for i, e in enumerate(e_s):
                    jump_prob = jump_dict.get(get_jump(j,i,l,m), 1/l)
                    translate_prob = translate_dict[e][f]
                    prob = jump_prob * translate_prob / sum_of_probs
                    co_counts[(e,f)] += prob
                    counts[e] += prob
                    jump_counts[get_jump(j,i,l,m)] += prob
                    pos_counts += prob

        print("Maximisation step {}".format(iteration))
        for e, f in co_counts:
            translate_dict[e][f] = co_counts[(e, f)] / counts[e]
        for jump in jump_counts:
            jump_dict[jump] = jump_counts[jump] / pos_counts
            
        #writing the iteration files in naacl for AER use
        alignments = align_all(validation, translate_dict, jump_dict, fname)
        ll = log_likelihood(data, translate_dict, jump_dict)
        aer = test("", alignments)
        aers.append(aer)
        lls.append(ll)
        print("AER {}, LL {}".format(aer, ll))
        
        # save models in between
        pickle.dump(translate_dict, open("translate_dicts/ibm2_em_t_epoch_{}.pickle".format(iteration + 1), 'wb'))
        pickle.dump(jump_dict, open("translate_dicts/ibm2_em_jump_epoch_{}.pickle".format(iteration + 1), 'wb'))
    
    # write aers and lls to file
    with open("ibm2_em_scores.txt", 'w') as f:
        f.write("AER")
        for aer in aers:
            f.write("{}\n".format(aer))
        f.write("\nLL")
        for ll in lls:
            f.write("{}\n".format(ll))
    return translate_dict, jump_dict

def get_jump(fre_pos, eng_pos, eng_len, fre_len):
    """Get jump.
    
    Args:
        fre_pos (int): position in French sentence
        eng_post (int): position in English sentence
        eng_len (int): length of English sentence
        fre_len (int): length of French sentence

    Returns:
        int: jump
    """
    equivalent_pos = int(math.floor(fre_pos * eng_len / fre_len))
    return eng_pos - equivalent_pos

# pretrained_t = pickle.load(open("epoch_12.pickle", 'rb'))

### 4. Train a model

In [9]:
ibm2_transdict, ibm2_jumpdict = EM_IBM2(
    training_data[:1000], validation_data,
    initialise_method = 'uniform', max_steps=10
)

Expectation step 0


100%|█████████████████████████████████████| 1000/1000 [00:03<00:00, 299.78it/s]


Maximisation step 0
AER 0.44572025052192066, LL -113561.99746145695
Expectation step 1


100%|█████████████████████████████████████| 1000/1000 [00:03<00:00, 317.66it/s]


Maximisation step 1
AER 0.39263157894736844, LL -85950.25559685814
Expectation step 2


...

Maximisation step 7
AER 0.4027777777777778, LL -49048.16477408836
Expectation step 8


100%|█████████████████████████████████████| 1000/1000 [00:03<00:00, 322.48it/s]


Maximisation step 8
AER 0.4027777777777778, LL -48470.21680337041
Expectation step 9


100%|█████████████████████████████████████| 1000/1000 [00:02<00:00, 388.95it/s]


Maximisation step 9
AER 0.3995726495726496, LL -48068.76398049057
